In [ ]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from multi_task_model.mtl import MTLInference

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

DATA_FOLDER = Path('../data/0_external/google-quest-challenge/')
MODEL_DIR = Path('../models/multitask-models/')
HF_MODEL_CARD = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'

In [ ]:
output_dir = MODEL_DIR / 'multitask-multilabel-model-finetuned-classification-layer-20230609'
tokenizer_dir = output_dir / 'tokenizer'
model_file = output_dir / 'mtl.bin'

In [ ]:
df_train = pd.read_csv(DATA_FOLDER / 'train.csv')
label_dict = dict([(k,v) for k, v in enumerate(df_train.category.unique())])

In [ ]:
def pred_raw(q, mtl_model):
    return mtl_model.predict(q)[0].squeeze().tolist()

def pred_index(p):
    pred = np.flatnonzero(np.array(p).__gt__(0.5)).tolist()
    if isinstance(pred, int):
        return [pred]
    else:
        return pred
  
def pred_label(p):
    return [label_dict[i] for i in p]

def predict(query, mtl_model):
    p_raw = pred_raw(query, mtl_model)
    p = pred_index(p_raw)
    return pred_label(p)

In [ ]:
mtl_model = MTLInference(tokenizer_dir, model_file, pretrained_model=False)

In [ ]:
query = 'how do multitask models work?'
predict(query, mtl_model)